In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model
from augmix import RandomAugMix

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
np.random.rand()

0.5621946804610382

In [8]:
HEIGHT = 137
WIDTH = 236

In [9]:
#import albumentations as albu
augmix = RandomAugMix(severity=3, width=3, alpha=1., p=1.)

def get_train_augs():
    #if np.random.rand() > 1.6:
    #    return RandAugment(n=2, m=27), 1
    #else:
    #    return augmix, 2
    return RandAugment(n=2, m=27)

In [10]:
#plt.imshow(x)

In [11]:
#np.arange(10).copy()

In [12]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
            #img = augs(image=img)['image']
            #if aug_type == 1:
        #    img = np.asarray(augs(Image.fromarray(img)))
            #elif aug_type == 2:
            #    img = augs(image=img)['image'].astype(np.uint8)
            #else:
            #    raise RuntimeError('error aug_type')
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [13]:
#train_loader, val_loader = get_train_val_loaders()

In [14]:
#for x in train_loader:
#    print(x)
#    break

# model

In [15]:
#import pretrainedmodels

In [16]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [17]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

In [18]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

In [19]:
class BengaliNet4(nn.Module):
    def __init__(self, backbone_name='se_resnext101_32x4d'):
        super(BengaliNet4, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.n_word = 1295
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant + self.n_word
        
        self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
        self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)
        
        self.num_p2_features = self.backbone.layer2[-1].se_module.fc2.out_channels
        self.num_p3_features = self.backbone.layer3[-1].se_module.fc2.out_channels
        self.p2_head = nn.Conv2d(self.num_p2_features, self.num_p2_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.p3_head = nn.Conv2d(self.num_p3_features, self.num_p3_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(self.num_p2_features * 4)
        self.bn3 = nn.BatchNorm2d(self.num_p3_features * 4)
        self.act2 = Swish()
        self.act3 = Swish()
        
        self.fc_aux1 = nn.Linear(self.num_p3_features * 4, self.num_classes)
        self.fc_aux2 = nn.Linear(self.num_p2_features * 4, self.num_classes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        for fc in [self.fc, self.fc_aux1, self.fc_aux2]:
            nn.init.zeros_(fc.bias.data)

        print('init model4')
        
    def features(self, x):
        x = self.backbone.layer0(x); #print(x.size())
        x = self.backbone.layer1(x); #print(x.size())
        x = self.backbone.layer2(x); p2 = x; p2 = self.p2_head(p2); p2 = self.bn2(p2); p2 = self.act2(p2) #print(x.size())
        x = self.backbone.layer3(x); p3 = x; p3 = self.p3_head(p3); p3 = self.bn3(p3); p3 = self.act3(p3) #print(x.size())
        x = self.backbone.layer4(x); #print(x.size())
        return x, p2, p3
        
    def logits(self, x, p2, p3):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = torch.flatten(x, 1)
        
        p2 = self.avg_pool(p2)
        p2 = torch.flatten(p2, 1)
        
        p3 = self.avg_pool(p3)
        p3 = torch.flatten(p3, 1)
        return self.fc(x), self.fc_aux1(p3), self.fc_aux2(p2)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        x, p2, p3 = self.features(x)
        x, logits_aux1, logits_aux2 = self.logits(x, p2, p3)

        return x, logits_aux1, logits_aux2

In [20]:
MODEL_DIR = './model4-ckps'
def create_model(args):
    model = BengaliNet4(args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [21]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [22]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [23]:
round(1/9, 6)

0.111111

In [24]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [25]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()
    #return loss3

In [26]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, outputs_aux1, outputs_aux2 = model(x)
            #avg_outputs = torch.mean(torch.stack([outputs, outputs_aux1, outputs_aux2], 0), 0)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [27]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [28]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [29]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [30]:
def rand_bbox_old(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [31]:
def rand_bbox(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [32]:
np.random.random()

0.48517551026158534

In [33]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [34]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [35]:
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        if r < 0.3:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            #img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.7:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, targets)
            loss_aux1 = criterion(outputs_aux1, targets)
            loss_aux2 = criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            img, targets = mixup(img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = mixup_criterion(outputs, targets)
            loss_aux1 = mixup_criterion(outputs_aux1, targets)
            loss_aux2 = mixup_criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1)), end='')


In [36]:
best_metrics = 0.

def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()
    

def train(args):
    model, model_file = create_model(args)
    model = model.cuda()
    
    #for param in model.backbone.parameters():
    #    param.requires_grad = False

    swa_model, _ = create_model(args)
    swa_model = swa_model.cuda()
    swa_model_file = model_file

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.start_epoch, args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                copy_model(swa_model, model)
                swa_n = 0
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save(swa_model, swa_model_file, val_loader, save=True)

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

In [37]:
args = Namespace()
args.backbone = 'se_resnext101_32x4d'
args.ckp_name = 'model4_se_resnext101_fold3_224.pth'

args.base_lr = 1e-4

args.num_epochs = 60
args.warmup_epochs = 5
args.start_epoch = 0

args.swa_start = 1000
args.swa_freq = 4

args.num_cycles = 100
args.batch_size = 440
args.val_batch_size = 1024
args.st_epochs = 10

args.beta = 1.0
args.cutmix_prob = 0.5

In [38]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=3)

(200840, 6)
(200840, 32332)
(160716, 6) (40124, 6)


In [39]:
train(args)

init model4
model file: ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth, exist: True
loading ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth...
init model4
model file: ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth, exist: True
loading ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth...

val: {'recall': 0.99568, 'recall_grapheme': 0.993924, 'recall_vowel': 0.996616, 'recall_consonant': 0.998254, 'recall_word': 0.992653, 'acc_grapheme': 0.993146, 'acc_vowel': 0.99686, 'acc_consonant': 0.997582, 'acc_word': 0.992498, 'loss_grapheme': 0.027386, 'loss_vowel': 0.017612, 'loss_consonant': 0.013515, 'loss_word': 0.027576}
CYCLE: 1
    0 | 0.000020 | 160600/160716 | 8.4128 | 6.9039 ||
val: {'recall': 0.99351, 'recall_grapheme': 0.990305, 'recall_vowel': 0.996005, 'recall_consonant': 0.997425, 'recall_word': 0.988899, 'acc_grapheme': 0.990255, 'acc_vowel': 0.996212, 'acc_consonant': 0.996287, 'acc_word': 0.98873

   20 | 0.000073 | 160600/160716 | 12.6315 | 7.3443 ||
val: {'recall': 0.993016, 'recall_grapheme': 0.989521, 'recall_vowel': 0.996378, 'recall_consonant': 0.996646, 'recall_word': 0.98837, 'acc_grapheme': 0.989208, 'acc_vowel': 0.996361, 'acc_consonant': 0.995888, 'acc_word': 0.988187, 'loss_grapheme': 0.047492, 'loss_vowel': 0.033285, 'loss_consonant': 0.028356, 'loss_word': 0.046543}
   21 | 0.000070 | 160600/160716 | 17.0000 | 6.9909 ||
val: {'recall': 0.994627, 'recall_grapheme': 0.992146, 'recall_vowel': 0.996667, 'recall_consonant': 0.997549, 'recall_word': 0.991286, 'acc_grapheme': 0.991551, 'acc_vowel': 0.99676, 'acc_consonant': 0.996536, 'acc_word': 0.991152, 'loss_grapheme': 0.035714, 'loss_vowel': 0.028704, 'loss_consonant': 0.023212, 'loss_word': 0.034285}
   22 | 0.000068 | 160600/160716 | 8.0090 | 7.4144 |||
val: {'recall': 0.994571, 'recall_grapheme': 0.992037, 'recall_vowel': 0.996608, 'recall_consonant': 0.997601, 'recall_word': 0.990367, 'acc_grapheme': 0.991427, 'ac

   41 | 0.000021 | 160600/160716 | 3.2032 | 5.6600 |||
val: {'recall': 0.994841, 'recall_grapheme': 0.992356, 'recall_vowel': 0.99655, 'recall_consonant': 0.998104, 'recall_word': 0.991304, 'acc_grapheme': 0.992174, 'acc_vowel': 0.99691, 'acc_consonant': 0.997184, 'acc_word': 0.991078, 'loss_grapheme': 0.030183, 'loss_vowel': 0.017648, 'loss_consonant': 0.013031, 'loss_word': 0.032839}
   42 | 0.000019 | 160600/160716 | 3.7560 | 7.0704 ||
val: {'recall': 0.994494, 'recall_grapheme': 0.992099, 'recall_vowel': 0.996181, 'recall_consonant': 0.997596, 'recall_word': 0.990653, 'acc_grapheme': 0.991277, 'acc_vowel': 0.996411, 'acc_consonant': 0.996436, 'acc_word': 0.990405, 'loss_grapheme': 0.036638, 'loss_vowel': 0.029291, 'loss_consonant': 0.022599, 'loss_word': 0.036438}
   43 | 0.000017 | 160600/160716 | 7.5272 | 6.1906 ||
val: {'recall': 0.994669, 'recall_grapheme': 0.992279, 'recall_vowel': 0.996462, 'recall_consonant': 0.997658, 'recall_word': 0.99046, 'acc_grapheme': 0.991601, 'acc_v

    2 | 0.000060 | 160600/160716 | 16.9800 | 6.1457 |
val: {'recall': 0.993546, 'recall_grapheme': 0.990673, 'recall_vowel': 0.995647, 'recall_consonant': 0.997188, 'recall_word': 0.989191, 'acc_grapheme': 0.990131, 'acc_vowel': 0.996162, 'acc_consonant': 0.996037, 'acc_word': 0.988984, 'loss_grapheme': 0.050674, 'loss_vowel': 0.043321, 'loss_consonant': 0.033717, 'loss_word': 0.046422}
    3 | 0.000079 | 160600/160716 | 0.1082 | 5.7816 ||
val: {'recall': 0.994508, 'recall_grapheme': 0.99202, 'recall_vowel': 0.996274, 'recall_consonant': 0.997719, 'recall_word': 0.990077, 'acc_grapheme': 0.991427, 'acc_vowel': 0.99666, 'acc_consonant': 0.99681, 'acc_word': 0.989881, 'loss_grapheme': 0.034263, 'loss_vowel': 0.016656, 'loss_consonant': 0.01309, 'loss_word': 0.038544}
    4 | 0.000098 | 160600/160716 | 7.7661 | 6.3727 ||
val: {'recall': 0.993118, 'recall_grapheme': 0.990082, 'recall_vowel': 0.996214, 'recall_consonant': 0.996093, 'recall_word': 0.98825, 'acc_grapheme': 0.989433, 'acc_vowe

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   41 | 0.000021 | 160600/160716 | 0.1711 | 6.4507 ||
val: {'recall': 0.995452, 'recall_grapheme': 0.993582, 'recall_vowel': 0.996698, 'recall_consonant': 0.997947, 'recall_word': 0.991536, 'acc_grapheme': 0.992648, 'acc_vowel': 0.996885, 'acc_consonant': 0.997109, 'acc_word': 0.991327, 'loss_grapheme': 0.030605, 'loss_vowel': 0.020579, 'loss_consonant': 0.016235, 'loss_word': 0.032595}
   42 | 0.000019 | 160600/160716 | 13.7321 | 6.5739 |
val: {'recall': 0.994493, 'recall_grapheme': 0.991957, 'recall_vowel': 0.996472, 'recall_consonant': 0.997586, 'recall_word': 0.989771, 'acc_grapheme': 0.991177, 'acc_vowel': 0.996685, 'acc_consonant': 0.996486, 'acc_word': 0.989557, 'loss_grapheme': 0.040118, 'loss_vowel': 0.028852, 'loss_consonant': 0.024275, 'loss_word': 0.040446}
   43 | 0.000017 | 160600/160716 | 0.1625 | 6.3324 ||
val: {'recall': 0.995106, 'recall_grapheme': 0.993205, 'recall_vowel': 0.996701, 'recall_consonant': 0.997314, 'recall_word': 0.991316, 'acc_grapheme': 0.992099, 'acc

   24 | 0.000063 | 160600/160716 | 0.2151 | 6.5917 ||
val: {'recall': 0.995077, 'recall_grapheme': 0.992798, 'recall_vowel': 0.996663, 'recall_consonant': 0.998048, 'recall_word': 0.991815, 'acc_grapheme': 0.992349, 'acc_vowel': 0.99676, 'acc_consonant': 0.997059, 'acc_word': 0.991676, 'loss_grapheme': 0.031272, 'loss_vowel': 0.022871, 'loss_consonant': 0.018231, 'loss_word': 0.030701}
   25 | 0.000060 | 160600/160716 | 0.2318 | 6.5634 |||
val: {'recall': 0.994484, 'recall_grapheme': 0.991988, 'recall_vowel': 0.996017, 'recall_consonant': 0.997941, 'recall_word': 0.990986, 'acc_grapheme': 0.991751, 'acc_vowel': 0.996611, 'acc_consonant': 0.99691, 'acc_word': 0.990853, 'loss_grapheme': 0.033002, 'loss_vowel': 0.019112, 'loss_consonant': 0.015692, 'loss_word': 0.035463}
   26 | 0.000058 | 160600/160716 | 0.2077 | 6.2640 ||
val: {'recall': 0.993863, 'recall_grapheme': 0.99135, 'recall_vowel': 0.995571, 'recall_consonant': 0.997181, 'recall_word': 0.989498, 'acc_grapheme': 0.990255, 'acc_v

KeyboardInterrupt: 

In [39]:
#for param in model.backbone.parameters():
#    param.requires_grad = False

In [40]:
#validate(nn.DataParallel(model), val_loader)

In [41]:
train(args)

init model4
model file: ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth, exist: True
loading ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth...
init model4
model file: ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth, exist: True
loading ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth...

val: {'recall': 0.995777, 'recall_grapheme': 0.994077, 'recall_vowel': 0.996846, 'recall_consonant': 0.99811, 'recall_word': 0.99307, 'acc_grapheme': 0.99347, 'acc_vowel': 0.997059, 'acc_consonant': 0.997682, 'acc_word': 0.993047, 'loss_grapheme': 0.0246, 'loss_vowel': 0.012441, 'loss_consonant': 0.008668, 'loss_word': 0.02692}
CYCLE: 1
    0 | 0.000015 | 160600/160716 | 1.9527 | 12.0129 ||
val: {'recall': 0.992213, 'recall_grapheme': 0.988496, 'recall_vowel': 0.995465, 'recall_consonant': 0.996394, 'recall_word': 0.98645, 'acc_grapheme': 0.988037, 'acc_vowel': 0.996012, 'acc_consonant': 0.995763, 'acc_word': 0.986317, 

   20 | 0.000109 | 160600/160716 | 12.8580 | 12.4901 |
val: {'recall': 0.992932, 'recall_grapheme': 0.989504, 'recall_vowel': 0.99572, 'recall_consonant': 0.996999, 'recall_word': 0.98799, 'acc_grapheme': 0.988486, 'acc_vowel': 0.996237, 'acc_consonant': 0.996137, 'acc_word': 0.987863, 'loss_grapheme': 0.091197, 'loss_vowel': 0.081313, 'loss_consonant': 0.059179, 'loss_word': 0.077822}
   21 | 0.000106 | 160600/160716 | 21.4262 | 11.8378 |
val: {'recall': 0.991404, 'recall_grapheme': 0.987282, 'recall_vowel': 0.995096, 'recall_consonant': 0.995958, 'recall_word': 0.984981, 'acc_grapheme': 0.986367, 'acc_vowel': 0.995788, 'acc_consonant': 0.995514, 'acc_word': 0.984772, 'loss_grapheme': 0.08523, 'loss_vowel': 0.068181, 'loss_consonant': 0.052766, 'loss_word': 0.078215}
   22 | 0.000102 | 160600/160716 | 18.5419 | 11.5148 |
val: {'recall': 0.992849, 'recall_grapheme': 0.98917, 'recall_vowel': 0.996185, 'recall_consonant': 0.996869, 'recall_word': 0.987154, 'acc_grapheme': 0.98861, 'acc_v

    2 | 0.000045 | 160600/160716 | 6.3126 | 11.7465 ||
val: {'recall': 0.992897, 'recall_grapheme': 0.989372, 'recall_vowel': 0.995471, 'recall_consonant': 0.997373, 'recall_word': 0.987215, 'acc_grapheme': 0.988685, 'acc_vowel': 0.996162, 'acc_consonant': 0.996361, 'acc_word': 0.98709, 'loss_grapheme': 0.072414, 'loss_vowel': 0.062011, 'loss_consonant': 0.048047, 'loss_word': 0.06711}
    3 | 0.000059 | 160600/160716 | 5.7804 | 11.6953 ||
val: {'recall': 0.992867, 'recall_grapheme': 0.989194, 'recall_vowel': 0.9957, 'recall_consonant': 0.997381, 'recall_word': 0.987, 'acc_grapheme': 0.988486, 'acc_vowel': 0.995813, 'acc_consonant': 0.996212, 'acc_word': 0.986791, 'loss_grapheme': 0.070728, 'loss_vowel': 0.055763, 'loss_consonant': 0.044861, 'loss_word': 0.068359}
    4 | 0.000074 | 160600/160716 | 20.0910 | 12.2234 |
val: {'recall': 0.993096, 'recall_grapheme': 0.98954, 'recall_vowel': 0.995603, 'recall_consonant': 0.9977, 'recall_word': 0.986765, 'acc_grapheme': 0.988461, 'acc_vowel'

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)